INSTALL
- FAISS - 
- Sent2Vec

In [1]:
import numpy as np
import faiss   

In [2]:
import os
import time
name = "TED-bert-"+ str(time.strftime("%Y%m%d-%H%M"))

!mkdir $name
path = os.getcwd() + "/" + name
os.chdir(path)

!pwd


os.chdir(path)


# import os
# os.chdir("/home/lw/clc_fairseq/TED-20210121-1335")

/home/chinh/clc_data/dataset/TED-bert-20210409-1708


In [3]:
!wget https://github.com/hoangtrungchinh/clc_data/raw/master/en_vi_iwslt.tar.gz
!mkdir data_bin
!tar -xvf  'en_vi_iwslt.tar.gz' -C 'data_bin'

--2021-04-09 17:08:08--  https://github.com/hoangtrungchinh/clc_data/raw/master/en_vi_iwslt.tar.gz
Resolving github.com (github.com)... 192.30.255.112
Connecting to github.com (github.com)|192.30.255.112|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/en_vi_iwslt.tar.gz [following]
--2021-04-09 17:08:09--  https://raw.githubusercontent.com/hoangtrungchinh/clc_data/master/en_vi_iwslt.tar.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7497739 (7,2M) [application/octet-stream]
Saving to: ‘en_vi_iwslt.tar.gz’

en_vi_iwslt.tar.gz  100%[===================>]   7,15M  8,05MB/s    in 0,9s    

2021-04-09 17:08:11 (8,05 MB/s) - ‘en_vi_iwslt.tar.gz’ saved [7497739/7497

In [4]:
# import file
import numpy as np
import glob

en_files = sorted(glob.glob("data_bin/*.en"))
vi_files = sorted(glob.glob("data_bin/*.vi"))
print(en_files)
print(vi_files)

lst_en = []
lst_vi = []

start = time.time()

for file in en_files:
  with open(file) as file_in:
    for line in file_in:
      lst_en.append(line)

for file in vi_files:
  with open(file) as file_in:
    for line in file_in:
      lst_vi.append(line)

print(len(lst_en))
print(len(lst_vi))

['data_bin/TED.en']
['data_bin/TED.vi']
103624
103624


In [5]:
# SPLIT TRAIN, TEST, VALID
from sklearn.model_selection import train_test_split
en_train, en_test_valid, vi_train, vi_test_valid =  train_test_split(lst_en, lst_vi, test_size=0.2, random_state=123)
en_valid, en_test, vi_valid, vi_test = train_test_split(en_test_valid, vi_test_valid, test_size=0.5, random_state=123)
print(len(en_train))
print(len(vi_train))
print(len(en_valid))
print(len(vi_valid))
print(len(en_test))
print(len(vi_test))
print(en_train[20])
print(vi_train[20])
print(en_valid[-1])
print(vi_valid[-1])
print(en_test[-1])
print(vi_test[-1])

82899
82899
10362
10362
10363
10363
And it's based on pizza .

Và nó dựa trên pizza .

You formulate a recipe based upon the factors you can control , whether it be the skills or the fertilizer or the plant choice .

Bạn xây dựng một công thức dựa trên những yếu tố có thể kiểm soát được , đó có thể là các kỹ năng hay phân bón hay chọn giống .

All of these are parts of his life and what he remembers .

Những điều này là một phần cuộc đời ông và là những gì ông còn nhớ



In [6]:
file_variable = [en_train,vi_train,en_valid,vi_valid,en_test,vi_test]
file_names = ["en_train","vi_train","en_valid","vi_valid","en_test","vi_test"]

for i in range(len(file_names)):
  with open(file_names[i], 'w') as f:
    for item in file_variable[i]:
      f.write(item)
!ls -al

total 30156
drwxrwxr-x 3 chinh chinh     4096 Thg 4   9 17:08 .
drwxrwxr-x 4 chinh chinh     4096 Thg 4   9 17:08 ..
drwxrwxr-x 2 chinh chinh     4096 Thg 4   9 17:08 data_bin
-rw-rw-r-- 1 chinh chinh   996149 Thg 4   9 17:08 en_test
-rw-rw-r-- 1 chinh chinh  8024744 Thg 4   9 17:08 en_train
-rw-rw-r-- 1 chinh chinh  1000856 Thg 4   9 17:08 en_valid
-rw-rw-r-- 1 chinh chinh  7497739 Thg 4   9 17:08 en_vi_iwslt.tar.gz
-rw-rw-r-- 1 chinh chinh  1327417 Thg 4   9 17:08 vi_test
-rw-rw-r-- 1 chinh chinh 10671354 Thg 4   9 17:08 vi_train
-rw-rw-r-- 1 chinh chinh  1330789 Thg 4   9 17:08 vi_valid


In [7]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('stsb-bert-large')

In [8]:
start = time.time()
srcfile = "en_train"
tarfile = "vi_train"
lst_srcfile = en_train
lst_tarfile = vi_train

vec_lst_srcfile = model.encode(lst_srcfile)
print('=== Ending, Total time (second): ', time.time() - start)

=== Ending, Total time (second):  829.9584522247314


# Calculate similarity

In [9]:
import faiss
vec_lst_srcfile_copy = vec_lst_srcfile.copy()
start = time.time()
index = faiss.IndexFlatIP(vec_lst_srcfile_copy.shape[1])
index.ntotal
faiss.normalize_L2(vec_lst_srcfile_copy)
index.add(vec_lst_srcfile_copy)
k = 6
distance, index = index.search(vec_lst_srcfile_copy, k)

print('=== Ending, Total time (second): ', time.time() - start)
print(len(distance))
print(len(index))

print(index)
print(distance)

=== Ending, Total time (second):  40.45434236526489
82899
82899
[[    0  7856 25736 48911 53083 22897]
 [    1 14961  5104 58108 31087 74921]
 [    2 75240 45075 11299 28894 60472]
 ...
 [82896  5715 12538 16082 67091 25738]
 [82897  2513 61211 72670 73811 55593]
 [82898 11856 65086 60052 80353 27362]]
[[0.99999994 0.6778243  0.6508336  0.64470613 0.643991   0.6407182 ]
 [1.0000001  0.5626349  0.56184876 0.5178615  0.5073554  0.4981301 ]
 [1.0000002  0.5638742  0.5589752  0.55712193 0.5544174  0.54987687]
 ...
 [1.0000001  0.64247674 0.64090955 0.6311184  0.6309517  0.62785524]
 [0.99999994 0.60968727 0.53996557 0.5366238  0.5316666  0.52403253]
 [1.         0.72527915 0.6619356  0.6608713  0.6514109  0.6387154 ]]


In [10]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial import distance as dt

count = 0
for i in range(len(lst_srcfile)):
    j = 1 #1 is index of biggest simmilarity
    score = distance[i][j]
    a = vec_lst_srcfile[i]
    b = vec_lst_srcfile[index[i][j]]
    
    dot_product = np.dot(a, b) # x.y
    norm_a = np.linalg.norm(a) #|x|
    norm_b = np.linalg.norm(b) #|y|
    sim_res = dot_product / (norm_a * norm_b)

    if score >=0.98 :
        count+=1
        print("==========================")
        for m in range(len(index[i])):
            print("--------", distance[i][m])
        print(i, index[i][j], distance[i][j], sim_res, 1-dt.cosine(a,b))
        print(lst_srcfile[i])
        print(lst_srcfile[index[i][j]])
        print("==========================")
print(count)

-------- 0.9999998
-------- 0.98917407
-------- 0.98574686
-------- 0.9138494
-------- 0.8890343
-------- 0.87741745
188 45518 0.98917407 0.9891742 0.9891742467880249
Well let me show you .

And let me show you .

-------- 0.9999997
-------- 0.9854067
-------- 0.9834658
-------- 0.9621835
-------- 0.9621484
-------- 0.9438308
399 26689 0.9854067 0.98540676 0.985406756401062
And that's how it works .

And this is how it works .

-------- 1.0000001
-------- 0.983349
-------- 0.85176545
-------- 0.8338403
-------- 0.82130104
-------- 0.81714076
631 72302 0.983349 0.9833489 0.9833488464355469
It doesn't stop there .

And it didn't stop there .

-------- 0.99999994
-------- 0.9864818
-------- 0.83901525
-------- 0.80810106
-------- 0.79492
-------- 0.7942251
1273 38927 0.9864818 0.9864817 0.9864816665649414
It's a huge amount of data .

And that's a huge amount of data .

-------- 0.9999998
-------- 0.98344564
-------- 0.98324734
-------- 0.92278963
-------- 0.8948689
-------- 0.8334975
160

-------- 0.9999999
-------- 0.9944241
-------- 0.96832544
-------- 0.831017
-------- 0.8215686
-------- 0.81250083
14305 42742 0.9944241 0.9944242 0.9944241642951965
And I know what you're thinking .

I know what you're thinking .

-------- 0.9999998
-------- 0.989125
-------- 0.9647316
-------- 0.8405967
-------- 0.80373114
-------- 0.79189557
14605 46518 0.989125 0.98912495 0.9891250133514404
And this is my house .

This is my house .

-------- 1.0
-------- 0.99098307
-------- 0.937317
-------- 0.9012357
-------- 0.9008919
-------- 0.8898796
14620 44724 0.99098307 0.9909829 0.9909828901290894
This one's my favorite .

That one's my favorite .

-------- 1.0
-------- 0.98303545
-------- 0.73523164
-------- 0.64676356
-------- 0.6101689
-------- 0.60841626
15201 65468 0.98303545 0.9830355 0.9830356240272522
So what about the poorest two billion ?

What about the poorest two billion here ?

-------- 0.9999997
-------- 0.9917793
-------- 0.61129963
-------- 0.59969956
-------- 0.59642524


-------- 0.9999998
-------- 0.9911268
-------- 0.9173049
-------- 0.8719381
-------- 0.8686941
-------- 0.86543244
24821 64214 0.9911268 0.9911269 0.9911270141601562
Now , look at this .

Now look at this .

-------- 1.0000001
-------- 0.9888907
-------- 0.9782272
-------- 0.9754277
-------- 0.91799766
-------- 0.8954951
25555 53581 0.9888907 0.98889065 0.9888906478881836
Let me show you how it works .

So let me show you how this works .

-------- 1.0000002
-------- 0.9841935
-------- 0.9831153
-------- 0.9402966
-------- 0.93978214
-------- 0.918883
25558 74986 0.9841935 0.9841937 0.9841936826705933
And I said , " Yes . "

So I said , " Yes . "

-------- 1.0000001
-------- 0.9926778
-------- 0.96442395
-------- 0.9203044
-------- 0.90218
-------- 0.8704195
25658 54396 0.9926778 0.99267775 0.9926778078079224
So , what do you do ?

So what do you do ?

-------- 1.0000001
-------- 0.98892796
-------- 0.72803396
-------- 0.6996853
-------- 0.69640946
-------- 0.686547
25934 70071 0.98892

-------- 1.0000001
-------- 0.9908009
-------- 0.8288484
-------- 0.81798667
-------- 0.81279874
-------- 0.79935247
42663 66756 0.9908009 0.99080086 0.9908008575439453
And that's what I want to talk to you a little bit about today .

And that's what I'd like to talk to you a little bit about today .

-------- 0.99999994
-------- 0.9944241
-------- 0.96315426
-------- 0.82166725
-------- 0.8176433
-------- 0.81214595
42742 14305 0.9944241 0.9944242 0.9944241642951965
I know what you're thinking .

And I know what you're thinking .

-------- 1.0000004
-------- 0.98574543
-------- 0.9571412
-------- 0.9416018
-------- 0.9366831
-------- 0.9245369
43072 12944 0.98574543 0.98574543 0.9857454299926758
And that's huge .

That's huge .

-------- 0.9999999
-------- 0.9893302
-------- 0.9539249
-------- 0.78366256
-------- 0.77934885
-------- 0.7681443
43955 10558 0.9893302 0.98933035 0.9893304705619812
And this is where you come in .

That is where you come in .

-------- 0.9999999
-------- 0.

-------- 0.99999976
-------- 0.9812691
-------- 0.8901028
-------- 0.8490231
-------- 0.7704791
-------- 0.7224143
60307 19816 0.9812691 0.981269 0.9812689423561096
Now , I want to tell you a quick story .

And for this one , I want to tell you a quick story .

-------- 1.0
-------- 0.9807109
-------- 0.70712507
-------- 0.6925688
-------- 0.677863
-------- 0.6692814
60383 53506 0.9807109 0.98071086 0.9807108640670776
This was reinforced by the political climate of the time .

This was reinforced by the political climate of the day .

-------- 0.99999994
-------- 0.9845474
-------- 0.95064694
-------- 0.9503211
-------- 0.94408965
-------- 0.9439449
60492 15409 0.9845474 0.98454726 0.9845473170280457
That's an amazing thing .

It's an amazing thing .

-------- 1.0000001
-------- 0.9849497
-------- 0.980566
-------- 0.9621835
-------- 0.94823325
-------- 0.93046117
60806 26689 0.9849497 0.98494977 0.9849497079849243
So this is how it works .

And this is how it works .

-------- 1.0
---

-------- 1.0
-------- 0.9922898
-------- 0.9689351
-------- 0.93978214
-------- 0.9347868
-------- 0.9271471
79443 16011 0.9922898 0.9922896 0.992289662361145
They said , " Yes . "

And they said , " Yes . "

-------- 0.99999994
-------- 0.9947727
-------- 0.93544275
-------- 0.921593
-------- 0.90183496
-------- 0.89952517
80147 15628 0.9947727 0.9947725 0.9947724938392639
Yes we can .

Well yes we can .

-------- 1.0
-------- 0.98218185
-------- 0.9306448
-------- 0.90160143
-------- 0.8724294
-------- 0.85987306
80411 29350 0.98218185 0.9821819 0.982181966304779
That's okay .

And that's okay .

-------- 1.0000001
-------- 0.9858227
-------- 0.9583551
-------- 0.9381074
-------- 0.90007675
-------- 0.8577199
80886 35362 0.9858227 0.98582256 0.9858226776123047
Well , why is that important ?

Now why is that important ?

-------- 0.99999994
-------- 0.99089074
-------- 0.97828436
-------- 0.9723052
-------- 0.96056443
-------- 0.9388316
80968 36277 0.99089074 0.99089074 0.990890800952

In [11]:
start = time.time()
saperate = " || "

src_label = "S"
emb_label = "E"
thres_arr = [0.8, 0.85, 0.9, 0.95]
for i in range(len(thres_arr)):
    threshold = thres_arr[i]
    out_file_write = open("en_train_EM_"+str(thres_arr[i]), 'w')
    out_file_write_score = open("en_train_EM_score_"+str(thres_arr[i]), 'w')
    out_file_write_factor = open("en_train_EM_factor_"+str(thres_arr[i]), 'w')
    num_gth_thres = 0

    for i in range(len(lst_srcfile)):
        j=5
        score = distance[i][1]
        content = ""
        content_label_arr = []
        if score >= threshold and score <0.999:
            num_gth_thres += 1
            best_simi_index = index[i][j]
            content = lst_srcfile[i].strip() + saperate + lst_tarfile[best_simi_index].strip()

            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label.append(emb_label)
            content_label = content_label + [emb_label] * len(lst_tarfile[best_simi_index].split())

            content_label_arr = " ".join(content_label)
        else:
            content = lst_srcfile[i].strip()
            content_label = [src_label] * len(lst_srcfile[i].split())
            content_label_arr = " ".join(content_label)
 
        out_file_write.writelines(content+ "\n")    
        out_file_write_factor.writelines(content_label_arr + "\n")
        out_file_write_score.writelines(str(score) + "\n")

    print('threshold =', threshold, "EM sentences =", num_gth_thres, "/", len(lst_srcfile), num_gth_thres*100/len(lst_srcfile), "%" )

    out_file_write.close()
    out_file_write_score.close()
    out_file_write_factor.close()

done = time.time()
elapsed = done - start
print('=== Ending, Total time (second): ', elapsed)

threshold = 0.8 EM sentences = 9307 / 82899 11.226914679308557 %
threshold = 0.85 EM sentences = 4833 / 82899 5.8299858864401255 %
threshold = 0.9 EM sentences = 2397 / 82899 2.891470343429957 %
threshold = 0.95 EM sentences = 884 / 82899 1.0663578571514736 %
=== Ending, Total time (second):  6.318448305130005
